In [ ]:
import pandas as pd
import nltk
import re

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
ps=PorterStemmer()
data=[]

In [ ]:
df=pd.read_csv('IMDB Dataset.csv')

In [ ]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [ ]:
for i in range(0,50000):
  review=df['review'][i]

  #remove uneccessary things
  review=re.sub('[^a-zA-Z]',' ',review )

  #Lower case text
  review=review.lower()

  #split the text
  review=review.split()

  #Stemming & Remove Stopwords
  review=[ps.stem(word) for word in review if not word in 
          set(stopwords.words('english'))]

  review=' '.join(review)
  data.append(review)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=2000)

In [ ]:
x=cv.fit_transform(data).toarray()
y=df.iloc[:,1:2].values

In [ ]:
y

array([['positive'],
       ['positive'],
       ['positive'],
       ...,
       ['negative'],
       ['negative'],
       ['negative']], dtype=object)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [ ]:
ct = ColumnTransformer([("one",OneHotEncoder(),[0])],remainder ="passthrough")
y = ct.fit_transform(y)

In [ ]:
y

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [ ]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [ ]:
model=Sequential()

In [ ]:
x_train.shape

(40000, 2000)

In [ ]:
model.add(Dense(2000,activation='relu'))
model.add(Dense(4000,activation='relu'))
model.add(Dense(2,activation='softmax'))
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(x_train,y_train,epochs=10)

Epoch 1/10
1250/1250 [==============================] - 119s 95ms/step - loss: 0.3429 - accuracy: 0.8579
Epoch 2/10
1250/1250 [==============================] - 118s 95ms/step - loss: 0.2124 - accuracy: 0.9147
Epoch 3/10
1250/1250 [==============================] - 119s 95ms/step - loss: 0.0580 - accuracy: 0.9795
Epoch 4/10
1250/1250 [==============================] - 120s 96ms/step - loss: 0.0238 - accuracy: 0.9918
Epoch 5/10
1250/1250 [==============================] - 119s 96ms/step - loss: 0.0158 - accuracy: 0.9957
Epoch 6/10
1250/1250 [==============================] - 119s 96ms/step - loss: 0.0170 - accuracy: 0.9951
Epoch 7/10
1250/1250 [==============================] - 120s 96ms/step - loss: 0.0123 - accuracy: 0.9962
Epoch 8/10
1250/1250 [==============================] - 120s 96ms/step - loss: 0.0109 - accuracy: 0.9971
Epoch 9/10
1250/1250 [==============================] - 120s 96ms/step - loss: 0.0133 - accuracy: 0.9958
Epoch 10/10
1250/1250 [==============================] 

In [ ]:
model.save('imdbmodel.h5')
import pickle
pickle.dump(cv, open("vectorizer.pickle", "wb"))
#/content/vectorizer.pickle

In [ ]:
text= "it is the best good plot"
text=re.sub('[^a-zA-Z]',' ',text)
text=text.lower()
text=text.split()
text=[ps.stem(word) for word in text if not word in set(stopwords.words('english'))]
text=' '.join(text)

In [ ]:
y_p1= model.predict(cv.transform([text]))
y_p1=y_p1>0.5
y_p1[0][0]

False

In [ ]:
if y_p1[0][0]=='True':
        pred = "Negative"
else:
        pred = "pos"

In [ ]:
pred

'pos'